In [78]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid')

# load modules related to this exercise
from educationmodel_3_Lise_1 import EducationModel
import estimate_1 as est 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [79]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [80]:
model = EducationModel()

In [ ]:
model.solve()

In [ ]:
sol = model.sol

In [ ]:
sol.d.min()

In [ ]:
par = model.par 
sol= model.sol

In [ ]:
from consav.linear_interp import interp_4d

In [ ]:
# give ability
Dad_educ = family_data.Dad_educ
Mom_educ = family_data.Mom_educ
Family_income = family_data.Family_income
Num_siblings = family_data.Num_siblings
Nuclear = family_data.Nuclear
Urban = family_data.Urban
South = family_data.South


experience = decision_data.exp 
school_time = decision_data.edu


d = decision_data.dummy

util_school  = par.delta0*Dad_educ + par.delta1*Mom_educ + par.delta2*Family_income + par.delta3*Num_siblings + par.delta4*Nuclear + par.delta5*Urban + par.delta6*South
ability_wage = par.gamma0_w*Dad_educ + par.gamma1_w*Mom_educ + par.gamma2_w*Family_income + par.gamma3_w*Num_siblings + par.gamma4_w*Nuclear + par.gamma5_w*Urban + par.gamma6_w*South
ability_employment = par.gamma0_e*Dad_educ + par.gamma1_e*Mom_educ + par.gamma2_e*Family_income + par.gamma3_e*Num_siblings + par.gamma4_e*Nuclear + par.gamma5_e*Urban + par.gamma6_e*South
# ability_wage
# ability_employment

school_time_index =school_time-6

# probability of staying in school given the state
# AND THE ORTHOGONAL TYPE
log_likelihood = 0

epsilon = 1e-10

probabilities = {0: par.p1, 1: par.p2, 2: par.p3, 3: par.p4, 4: par.p5, 5: par.p6}
# Iterate over N and T
for i in range(par.N):
    for t in range(par.T):
        # Iterate through all choices including 0
        for x in range(1, 6):

            experience_i = experience[i + par.N * t]
            for i in range(t):
                #if experience_i equal to nan 
                if np.isnan(experience_i):
                    print('experience_i is nan')
                    experience_i = experience[i + par.N *(t-1)]
            
            if np.isnan(experience_i):
                print('experience_i is still nan')
                experience_i = 0

            sol_x = sol.d[t, x, :, :, :, school_time_index[i + par.N * t], :]
            lik_pr_x = interp_4d(
                par.nuw_fix_grid,
                par.nue_fix_grid,
                par.util_sch_fix_grid,
                par.experience_grid,
                sol_x,
                ability_wage[i],
                ability_employment[i],
                util_school[i],
                experience_i
            )
            clamped_lik_pr_x = np.clip(lik_pr_x, epsilon, 1-epsilon)

            if np.isnan(clamped_lik_pr_x).any() or np.isinf(clamped_lik_pr_x).any():
                print(f'NaN or Inf encountered at i={i}, t={t}, x={x}')
                print(f'clamped_lik_pr_x: {clamped_lik_pr_x}')
                print(f'choice_prob_x: {choice_prob_x}')
                clamped_lik_pr_x = 0.1
                continue

            choice_prob_x = clamped_lik_pr_x * d[i + par.N * t] + (1 - clamped_lik_pr_x) * (1 - d[i + par.N * t])
            #if choice_prob_x < 0:
            #    print('choice_prob_x <= 0, i, t:', i, t)
            #    print('choice_prob_x:', choice_prob_x)
            #    log_likelihood += -np.inf
            #else:

            if np.isnan(choice_prob_x).any() or np.isinf(choice_prob_x).any():
                print(f'NaN or Inf encountered at i={i}, t={t}, x={x}')
                print(f'clamped_lik_pr_x: {clamped_lik_pr_x}')
                print(f'choice_prob_x: {choice_prob_x}')
                continue

            if x in probabilities:
                log_likelihood += probabilities[x] * np.log(choice_prob_x)
            #print('log_likelihood =', log_likelihood)

print('log_likelihood =', log_likelihood)


In [ ]:
sol.d.mean()

In [ ]:
sol.d[0,5,14,:]

In [ ]:
sol.d

## Estimate

In [ ]:
par = model.par

In [ ]:
print(f'Max util_school: {util_school.max()}, Min util_school: {util_school.min()}')
print(f'Max ability_wage: {ability_wage.max()}, Min ability_wage: {ability_wage.min()}')
#print(f'Max ability_employment: {ability_employment.max()}, Min ability_employment: {ability_employment.min()}')


In [83]:
estimate_ = est.estimate_class()
family_data, decision_data = estimate_.read_data()

In [84]:
decision_data

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage
0,1,1980,1,13,0,0,0.0,NaN
1,2,1980,1,9,0,0,0.0,NaN
2,3,1980,1,10,0,0,0.0,NaN
3,4,1980,1,9,0,0,0.0,NaN
4,5,1980,1,13,0,0,0.0,NaN
...,...,...,...,...,...,...,...,...
17095,1706,1989,0,10,1,0,NaN,NaN
17096,1707,1989,0,12,1,0,NaN,NaN
17097,1708,1989,0,13,1,0,NaN,NaN
17098,1709,1989,0,12,1,0,NaN,NaN


In [ ]:
decision_data

In [ ]:
delta7= np.linspace(-5,5,5)
log_lik = np.nan + np.zeros(5)

In [ ]:
# Problemet er her, at vi rammer noget negativt. 
for i in range(5):
    pnames = ['delta7']
    theta = np.array([delta7[i]])
    log_lik[i]=estimate_.ll(theta,model, family_data, decision_data, pnames)

In [ ]:
decision_data[(decision_data['id'] == 8)]

In [53]:
filtered_decision_data = decision_data[(decision_data['dummy_school'] == 1)
                                        & (decision_data['dummy_work'] == 1)]

In [54]:
filtered_decision_data

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage


In [57]:
decision_data_1274 = decision_data[decision_data['id'] == 1274]
decision_data_1274

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage
1273,1274,1980,1,10,0,0,0.00,9.68
2983,1274,1981,0,11,0,1,1.00,NaN
4693,1274,1982,0,11,0,1,1.73,10.84
6403,1274,1983,0,11,0,1,2.10,11.39
8113,1274,1984,0,11,0,1,2.44,10.92
9823,1274,1985,0,11,0,1,2.54,10.93
11533,1274,1986,0,11,0,1,3.19,9.60
13243,1274,1987,0,11,1,0,3.52,NaN
14953,1274,1988,0,11,0,1,3.52,6.63
16663,1274,1989,0,11,0,1,4.00,5.85


In [58]:
decision_data_1028 = decision_data[decision_data['id'] == 1028]
decision_data_1028

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage
1027,1028,1980,1,10,0,0,0.00,NaN
2737,1028,1981,0,11,1,0,0.00,NaN
4447,1028,1982,1,11,0,0,0.00,NaN
6157,1028,1983,1,12,0,0,0.00,NaN
7867,1028,1984,1,13,0,0,0.00,NaN
9577,1028,1985,1,14,0,0,0.00,NaN
11287,1028,1986,1,15,0,0,0.00,NaN
12997,1028,1987,0,16,0,1,0.00,NaN
14707,1028,1988,0,16,0,1,1.00,4.03
16417,1028,1989,0,16,0,1,1.79,9.80


In [59]:
filtered_decision_data

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage


In [ ]:
delta7

In [ ]:
log_lik

In [ ]:
log_lik

In [ ]:
pnames = ['delta7']

In [ ]:
model = est.updatepar(model,pnames,theta)

In [ ]:
def updatepar(par,parnames, parvals):
    """ Update parameters """
    for i,parname in enumerate(parnames):
        parval = parvals[i]
        setattr(par,parname,parval)
    return par

In [ ]:
par = model.par

In [ ]:
updatepar(model.par, pnames, [7])

In [ ]:
par.delta7

In [ ]:
model.par.delta7

In [ ]:
fig = plt.figure(figsize=(6,4),dpi=100)

ax = fig.add_subplot(1,1,1)
ax.set_title('Log likelihood')

ax.plot(delta7,log_lik)

ax.set_xlabel('delta7')
ax.set_ylabel('log likelihood')


fig.tight_layout()

In [61]:
pnames = ['delta7']
bounds = [(None,None)]
theta0 = [0]    

In [62]:
estimate_.estimate(model, family_data, decision_data, pnames, theta0, bounds)

delta7 0.0
model solved
log_likelihood = -19553.482532554408
delta7 1.4901161193847656e-08
model solved
log_likelihood = -19553.482671947033
delta7 -1.0
model solved
log_likelihood = -15652.5032946093
delta7 -1.0000000149011612
model solved
log_likelihood = -15652.503306127677
delta7 -0.9236743689003888
model solved
log_likelihood = -15615.68930566305
delta7 -0.92367438380155
model solved
log_likelihood = -15615.68930876591
delta7 -0.8955325836956147
model solved
log_likelihood = -15612.739796031261
delta7 -0.8955325985967759
model solved
log_likelihood = -15612.739797371967
delta7 -0.8741213561344217
model solved
log_likelihood = -15613.271306176162
delta7 -0.8741213710355829
model solved
log_likelihood = -15613.271304737771
delta7 -0.8852032718978103
model solved
log_likelihood = -15612.609693215541
delta7 -0.8852032867989715
model solved
log_likelihood = -15612.60969257352
delta7 -0.8941373449256989
model solved
log_likelihood = -15612.687604646755
delta7 -0.89413735982686
model sol

In [ ]:
# delta 7 = -0.8883146062919914

In [81]:
pnames = ['delta7','q1','q2','q3','q4']
theta0 = [0,0,0,0,0]
bounds = [(None,None),(None,None),(None,None),(None,None),(None,None)] 

In [82]:
thetahat = estimate_.estimate(model, family_data, decision_data, pnames, theta0)

delta7 0.0
q1 0.0
q2 0.0
q3 0.0
q4 0.0
model solved
log_likelihood = -30440.824432444035
delta7 1.4901161193847656e-08
q1 0.0
q2 0.0
q3 0.0
q4 0.0
model solved
log_likelihood = -30440.82469249813
delta7 0.0
q1 1.4901161193847656e-08
q2 0.0
q3 0.0
q4 0.0
model solved
log_likelihood = -30440.824319058913
delta7 0.0
q1 0.0
q2 1.4901161193847656e-08
q3 0.0
q4 0.0
model solved
log_likelihood = -30440.824470380496
delta7 0.0
q1 0.0
q2 0.0
q3 1.4901161193847656e-08
q4 0.0
model solved
log_likelihood = -30440.824471436386
delta7 0.0
q1 0.0
q2 0.0
q3 0.0
q4 1.4901161193847656e-08
model solved
log_likelihood = -30440.82446897438
delta7 -0.9020748135082455
q1 0.39330995039003075
q2 -0.13159387530145844
q3 -0.13525654528052616
q4 -0.1267163474070208
model solved
log_likelihood = -18420.836531776393
delta7 -0.9020747986070843
q1 0.39330995039003075
q2 -0.13159387530145844
q3 -0.13525654528052616
q4 -0.1267163474070208
model solved
log_likelihood = -18420.836567128918
delta7 -0.9020748135082455
q1 0

In [ ]:
h = lambda theta: estimate_.ll(theta,model, family_data, decision_data, pnames)[0]
grad = estimate_.centered_grad(h,thetahat)


## Simulate

In [ ]:
model.simulate()

In [ ]:
sim=model.sim

In [ ]:
# Step 1: Check Inputs
print("school_time:", school_time)
print("experience:", experience)
assert not np.isnan(school_time).any(), "school_time contains NaN values"
assert not np.isnan(experience).any(), "experience contains NaN values"

# Step 2: Inspect Initialization and Updates
print("Initialization:")
print("Initial school_time:", sim.school_time_init[i])
print("Initial experience:", sim.experience_init[i])

# Inside the simulation loop
print("Updating school_time and experience:")
print("Updated school_time:", sim.school_time[i, t])
print("Updated experience:", sim.experience[i, t])

# Step 3: Debug wage Calculation
wage_value = self.wage(school_time_index, int(sim.experience[i, t]))
print("Wage value:", wage_value)

# Step 4: Review logwage and logestar Functions
# Check for NaN values in logwage and logestar calculations
logwage_value = self.logwage(school_time_index, int(sim.experience[i, t]))
logestar_value = self.logestar(school_time_index, int(sim.experience[i, t]))
print("logwage:", logwage_value)
print("logestar:", logestar_value)


In [ ]:
mean_d_vectors =[]
mean_school_time_vectors = []
mean_wage_vectors = []
mean_experience_vectors = []

In [ ]:
start = 0 
for i in range(model.par.Nfix):
    end = start + model.par.block_length
    
    # Extract slices
    d_slice = sim.d[start:end]
    school_time_slice = sim.school_time[start:end]
    wage_slice = sim.wage[start:end]
    experience_slice = sim.experience[start:end]

    # Check if slices are empty or contain only NaNs
    if np.isnan(d_slice).all():
        mean_d = np.nan  # or another default value
    else:
        mean_d = np.nanmean(d_slice, axis=0)

    if np.isnan(school_time_slice).all():
        mean_school_time = np.nan  # or another default value
    else:
        mean_school_time = np.nanmean(school_time_slice, axis=0)

    if np.isnan(wage_slice).all():
        mean_wage = np.nan  # or another default value
    else:
        mean_wage = np.nanmean(wage_slice, axis=0)

    if np.isnan(experience_slice).all():
        mean_experience = np.nan  # or another default value
    else:
        mean_experience = np.nanmean(experience_slice, axis=0)

    # Append the computed means to the lists
    mean_d_vectors.append(mean_d)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience_vectors.append(mean_experience)

    # Update the start index for the next block
    start = end


In [ ]:
start = 0 
for i in range(model.par.Nfix):
    end = start + model.par.block_length
    mean_d = sim.d[start:end].mean(axis=0)
    mean_school_time = sim.school_time[start:end].mean(axis=0)
    mean_wage = sim.wage[start:end].mean(axis=0, skipna = True)
    mean_d_vectors.append(mean_d)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience = sim.experience[start:end].mean(axis=0)
    mean_experience_vectors.append(mean_experience)
    start = end 


In [ ]:
plot_vectors = [mean_d_vectors, mean_school_time_vectors, mean_wage_vectors, mean_experience_vectors]
vector_names = ["Mean schooling choice", "Mean school time", "Mean wage", "Mean experience"]

In [ ]:
for vector, name in zip(plot_vectors, vector_names):
    for i in range(model.par.Nfix):
        plt.plot(vector[i], label=f"{name} for type {i}")
    plt.legend()
    plt.xlabel("Time period")
    plt.ylabel(name)
    plt.title(f"{name} for different types over time")
    plt.show()

## Andet